Import all the packages needed. You may need to download the programs using pip. 

In [ ]:
import pyabf
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd 
import os 
import csv

Path to directory with your ABF files. Change the path to your directory as appropriate. Include only Current Step protocols. This code is wriiten for current steps starting at -100pA increasing by 10pA each step. The current input starts at 0.025s and runs for 0.5 seconds. You may need to adjust other parts of the code to account for diiferent step protocols.

In [ ]:
directory = "path/to/your/directory"
directory

In [ ]:
current_directory = os.getcwd()
current_directory

Add all your ABF files into the directory prior to running code. Make sure you change the directory name to your directory. 

In [ ]:
#Load Abf files
directory = "your_directory"
abf_files = [f for f in os.listdir(directory) if f.endswith(".abf")]
abf_files #print names of abf files in directory

Initialize a DataFrame with the features the code is extracting. 

In [ ]:
results = pd.DataFrame(columns=["File", "RMP", "SS", "deltaSS", "Sag", "R_in", "rebound", "rheobase", "FSL"])
results

Set time ranges for each part of current step protocol. Adjust as needed if you use a different current step protocol. 

In [ ]:
x_range1 = (0, 0.025) #the time before current input
x_range2 = (0.4, 0.499) #the last 100ms of the current input
x_range3 = (0.03, 0.3) #the beggining of the current input
x_range4 = (0.525, 1.025) #the time after current input 
x_range5 = (0.025, 0.525) #the time range for the current input 

The 'for' loop below goes through each ABF file, extracts the features, and adds the data to the dataframe. 

In [ ]:
for abf_file in abf_files: #Loop through abf files
    abf_file_path = os.path.join(directory, abf_file)

    abf=pyabf.ABF(abf_file_path) #Open the abf file

    sweep_number = 0
    abf.setSweep(sweep_number)

    #calculate RMP of the first sweep, generally -100pA
    x_min1, x_max1 = x_range1
    rmp = abf.sweepY[(abf.sweepX >= x_min1) & (abf.sweepX <= x_max1)].mean()

    #calculate Steady State voltage for thefirst sweep
    x_min2, x_max2 = x_range2
    ss = abf.sweepY[(abf.sweepX >= x_min2) & (abf.sweepX <= x_max2)].mean()
    
    #calculate the lowest volatge at the beggining of the first sweep 
    x_min3, x_max3 = x_range3
    minval = abf.sweepY[(abf.sweepX >= x_min3) & (abf.sweepX <= x_max3)].min()

    #calculate the difference between Steady State and RMP 
    delta_ss = ss-rmp

    #calculate Sag percentage
    sag = ((minval-ss)/delta_ss)*100

    #calculate the current input of the first sweep
    time_point = 0.4

    data_index = int(time_point * abf.dataRate)
    
    I = abf.sweepC[data_index]

    #calculatue input resistance
    R_in = delta_ss / I *1000

    # Define the range of sweeps you want to check for rebound firing (first 10 sweeps)
    start_sweep = 0
    end_sweep = 9

    # Initialize a variable to store spike detection result
    spike_detected = 0

    # Iterate through the specified sweeps
    for sweep_number in range(start_sweep, end_sweep + 1):
        # Set the sweep number
        abf.setSweep(sweepNumber=sweep_number)

        # Extract the voltage values for the last 0.5 seconds
        voltage_last_half_second = abf.sweepY[(abf.sweepX >= x_range4[0]) & (abf.sweepX <= x_range4[1])]

        # Check for a spike/rebound firing (for simplicity, checking if the maximum voltage is above a certain threshold)
        spike_threshold = 5  # Replace with your threshold value
        if np.max(voltage_last_half_second) > spike_threshold:
            spike_detected = 1
            break  # Exit the loop if a spike/rebound firing is detected

    # Initialize variables for Rheobase calculation
    rheobase = np.nan  # Default to NaN if no Rheobase is found
    fsl = np.nan  # Default to NaN if no spike is detected

    # Set the starting sweep number to 11, the first sweep with positive current input
    start_sweep = 11

    # Iterate through sweeps to find Rheobase
    for sweep_number in range(start_sweep, abf.sweepCount):
        abf.setSweep(sweep_number)

        # Extract the voltage values for x_range5
        voltage_range5 = abf.sweepY[(abf.sweepX >= x_range5[0]) & (abf.sweepX <= x_range5[1])]
        # Check for a spike (checking if any voltage value exceeds the threshold)
        spike_threshold_rheo = 10  # Set spike threshold for rheobase to 5 mV
        if np.any(voltage_range5 > spike_threshold_rheo):
            # Set Rheobase as the current at x_range5
            rheobase_time_point = 0.3  # Adjust as needed
            rheobase = abf.sweepC[int(rheobase_time_point * abf.dataRate)]
        
            # Calculate FSL (time from the start of the sweep to the first spike)
            fsl = abf.sweepX[np.argmax(voltage_range5)] * 1000
            break  # Exit the loop if Rheobase is found
            
    sweep_result = pd.DataFrame({
        "File": abf_file, 
        "RMP": rmp, 
        "SS": ss, 
        "deltaSS": delta_ss, 
        "Sag": sag, 
        "R_in": R_in,
        "rebound": spike_detected,
        "rheobase": rheobase,
        "FSL": fsl}, 
            index=[0]
                          )
    results = pd.concat([results, sweep_result], axis=0, ignore_index=True)

print (results)

Save your DataFrame as a CSV. Make sure to change the filename to suit your needs. 

In [ ]:
file_name = "file_name.csv"
results.to_csv(file_name, index=False)